In [1]:
pip install -U sentence-transformers rank_bm25

^C
Note: you may need to restart the kernel to use updated packages.


In [2]:
import json
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import gzip
import os
import torch
import pandas as pd

In [ ]:
import csv
from csv import QUOTE_MINIMAL, DictWriter, QUOTE_NONE

bi_encoder = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
bi_encoder.max_seq_length = 256     #Truncate long passages to 256 tokens
top_k = 32                          #Number of passages we want to retrieve with the bi-encoder

#The bi-encoder will retrieve 100 documents. We use a cross-encoder, to re-rank the results list to improve the quality
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

# As dataset, we use Simple English Wikipedia. Compared to the full English wikipedia, it has only
# about 170k articles. We split these articles into paragraphs and encode them with the bi-encoder

train_df = pd.read_csv('D:\Github\AI\\train.json')
eval_df = pd.read_csv('D:\Github\AI\eval.json', header = None, delimiter="\t", quoting= csv.QUOTE_NONE, encoding='utf-8')
passages = train_df['context']

print("Passages:", len(passages))

# We encode all passages into our vector space. This takes about 5 minutes (depends on your GPU speed)
corpus_embeddings = bi_encoder.encode(passages, convert_to_tensor=True, show_progress_bar=True)

Passages: 10570


Batches:   0%|          | 0/331 [00:00<?, ?it/s]